# Benchmarks

We consider as benchmark models the discrete Lévy distribution and the bi-exponential distribution, given by equations

$$\Pr(L)=\zeta^{-1}_{(1+\beta, 1)} L^{-1-\beta}\,,$$ and $$\Pr(L) = \sum_{i=1,2} \omega_i (1-e^{-1/d_i}) e^{-(L-1)/d_i} \, ,$$ respectively, where $\zeta_{(1+\beta, 1)}=\sum_{\ell=0}^\infty (\ell+1)^{-1-\beta}$ is the Riemann zeta function, $d_i$ are length scales and the mode weights satisfy $\sum_{i=1,2} \omega_i=1$.

We transform the step length distributions into policies with Eq. (5), which is implemented in the code with the method ``policy_from_distr``.

In [ ]:
from rl_opts.utils_distributions import policy_from_distr, powerlaw, double_exp

#get policy from benchmark model
policy = policy_from_distr(parameters, max_length, model)

This method inputs (i) the parameter/s of the model, which is the exponent $\beta$ for the Lévy distribution and $d_1$, $d_2$, $\omega_1$ for the bi-exponential; (ii) the maximum value of step counter; and (iii) the model (either 'powerlaw' or 'double_exp').

We employ the library ``Tune`` for parameter optimization, which allows us to optimize the average search efficiency over a number a walks ``mean_eff`` with respect to the model parameters.

## CHANGE:
The function ``mean_eff`` to optimize is computed via ``average_search_efficiency``, and then reported to ``tune``. All this

In [1]:
from rl_opts.learn_and_bench import average_search_efficiency

The efficiency of each walk is again computed with ``walk_from_policy`` (see also tutorial on learning), but in this case, the policy is obtained from the benchmark distribution. Note that we save the efficiency achieved in each walk to be able to later retrieve the mean and the SEM.

In this case, the input config is a dictionary with the parameter ranges that the optimization algorithm will consider. 

The parameters of that dictionary are described below:

Model (we input parameter ranges here):

- `d_int` : small scale ($d_1$, first mode) of the bi-exponential distribution \
- `d_ext` : large scale ($d_2$, second mode) of the bi-exponential distribution \
- `p` : weight of the first mode ($\omega_1$) in the bi-exponential distribution \
- `beta` : exponent of the Lévy distribution \
- `model` : model description (either 'powerlaw' or 'double_exp')

Walks (we input a single value that is fixed throughout the optimization):

- `time_ep` : number of (small, $d=1$) steps per walk. We choose the same value for the benchmarks as for the episodes in the RL training \
- `n` : number of walks (also referred to as agents in the code, but there is no relation to RL agents)

Environment (we input a single value that is fixed throughout the optimization):

- `lc` : cutoff length \
- `Nt` : number of targets \
- `L` : world size \
- `r` : target detection radius \
- `destructive` : whether targets are destructive or not (always set to False) 

Once we define the parameter ranges, we choose the optimization algorithm. Among the different possibilities that ``Tune`` offers, we chose Grid Search for the Lévy distribution and Bayesian Optimization for the bi-exponential distributions.

#### Example

Let us take the example with $l_\textrm{c}=3$.

For the Lévy distribution, the config dictionary looks like:

In [ ]:
config = {'d_int': None,
          'd_ext': None,
          'p': None,
          'beta': tune.grid_search(np.linspace(0.01,1.,20)), 
          'model': 'powerlaw',
          'time_ep': 20000,
          'n': 10000,
          'lc': 3.0,
          'Nt': 100,
          'L': 100,
          'r': 0.5,
          'destructive': False
          }

We do a grid search over 20 parameters, linearly spaced in the interval $[0.01, 1]$. Parameters that correspond to the other model are set to 'None'.

Then, we initialize the tuner, which by default does a grid search over the input parameters.

In [ ]:
from ray import tune

tuner = tune.Tuner(average_search_efficiency,
                   tune_config=tune.TuneConfig(num_samples=1),
                   param_space=config)

And we run the algorithm:

In [ ]:
result_grid = tuner.fit()

For the bi-exponential distribution, the config dictionary looks like:

In [ ]:
config = {'d_int': tune.uniform(0.00001, 20.0),
          'd_ext': 100.0,
          'p': tune.uniform(0.0, 1.0),
          'beta': None,
          'model': 'double_exp',
          'time_ep': 20000,
          'n': 10000,
          'lc': 3.0,
          'Nt': 100,
          'L': 100,
          'r': 0.5,
          'destructive': False
          }

In this case, since we choose a Bayesian optimization method, we do not specify the parameters to try, but just the ranges. For the small scale, we consider a range that is of the order of the scale of $l_\textrm{c}$. We fix the value for $d_2$ to further guide the search and make it more time efficient. We do the search with $d_2=100$, which is the scale of the average distance between targets, and with $d_2=10^5$. Again, the parameter $\beta$ that corresponds to the other model is set to 'None'. 

We first initialize the Bayesian optimization method, and then the tuner.

In [ ]:
from ray import tune
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search import ConcurrencyLimiter

bayesopt = BayesOptSearch(metric="mean_eff", mode="max")
bayesopt = ConcurrencyLimiter(bayesopt, max_concurrent=3)
tuner = tune.Tuner(average_search_efficiency, 
                   tune_config=tune.TuneConfig(search_alg=bayesopt, num_samples=100), 
                   param_space=config)

Note that we limit the number of concurrent processes to 3, so that the method can update itself more times within the 100 samples.

And we run it:

In [ ]:
result_grid = tuner.fit()

#### Results

The results are saved as a panda dataframe in the folder 'results/benchmark_models/'. We further identify each run of the algorithm with the parameter `run`, in case there is more than one run per model (e.g. for the bi-exponential, we run it twice, for each value of $d_2$).

In [ ]:
run = '0'
results_path = 'results/benchmark_models/' + config['model'] + '/'+ str(config['lc']) + '/run_'+run+'/'

results_df = result_grid.get_dataframe()
results_df.to_csv(results_path+'df'+run+'_'+config['model']+'_lc_'+str(config['lc'])+'.csv')

To retrieve these results, run:

In [ ]:
import pandas as pd

results_df = pd.read_csv(results_path+'df'+run+'_'+config['model']+'_lc_'+str(config['lc'])+'.csv')

In addition, the model parameters that achieved the best performance, together with the corresponding mean search efficiency, and its standard error of the mean (which is the only value that is not contained in the dataframe results_df), can be obtained by running the method `get_opt`:

In [ ]:
from rl_opts.utils import get_opt

main_path = 'results/benchmark_models/'
lc = 3.0 #note that this notation has to match with how you have input it in the config dictionary
model = 'powerlaw'
run = '0'

mean_eff, sem, parameters = get_opt(main_path, lc, model, run)

The config dictionaries can also be saved and retrieved:

In [ ]:
config_path = 'configurations/benchmark_models/'

np.save(config_path+'config_'+config['model']+'_lc_'+str(config['lc'])+'_run_'+run+'.npy', config)
np.load(config_path+'config_'+config['model']+'_lc_'+str(config['lc'])+'_run_'+run+'.npy', allow_pickle=True).item()